# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import pandas as pd
import numpy as np
import pickle

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\N12667\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def load_data(dbfilepath, tablename):
    '''
    input: (
        dbfilepath: path to database
        tablename: name of table to fetch
            )
    Loads data from sqlite database 
    output: (
        X: features dataframe
        y: target dataframe
        category_names: names of targets
        )
    '''
    engine = create_engine(f'sqlite:///{dbfilepath}')
    df = pd.read_sql_table(tablename, engine)
    X = df['message']
    Y = df.iloc[:, 4:]
    category_names = Y.columns
    return X, Y, category_names

X, Y, category_names = load_data(dbfilepath='DisasterResponse.db', tablename='messages')

### 2. Write a tokenization function to process your text data

In [198]:
def tokenize(text, remove_stopwords=True):

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    stopw = set(stopwords.words('english')) if remove_stopwords else []

    clean_tokens = [lemmatizer.lemmatize(token).lower().strip() for token in tokens if token not in stopw]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [199]:
pipeline = Pipeline([
    ('vect' , CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10)))      
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [200]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
Y_pred = pipeline.predict(X_test)

In [230]:
def display_results(Y_test, Y_pred, model=None):

    target_names = Y_test.columns.values
    
    # Calculate the accuracy for each of them.
    for i in range(len(category_names)):
        print(f'Category:, {category_names[i]} \n {classification_report(Y_test.iloc[:, i].values, Y_pred[:, i])}')
        print(f'Accuracy of {category_names[i]}: {accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i]):.2f}')
        
    if model:
        print(f'\nBest Parameters: {model.best_params_}')
                                
display_results(Y_test, Y_pred)

Category:, related 
               precision    recall  f1-score   support

           0       0.71      0.35      0.47      1873
           1       0.82      0.96      0.88      5934
           2       0.67      0.07      0.12        58

   micro avg       0.80      0.80      0.80      7865
   macro avg       0.73      0.46      0.49      7865
weighted avg       0.79      0.80      0.78      7865

Accuracy of related: 0.80
Category:, request 
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      6533
           1       0.83      0.47      0.60      1332

   micro avg       0.89      0.89      0.89      7865
   macro avg       0.87      0.72      0.77      7865
weighted avg       0.89      0.89      0.88      7865

Accuracy of request: 0.89
Category:, offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

   micro avg   

Accuracy of buildings: 0.95
Category:, electricity 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      7697
           1       1.00      0.02      0.04       168

   micro avg       0.98      0.98      0.98      7865
   macro avg       0.99      0.51      0.51      7865
weighted avg       0.98      0.98      0.97      7865

Accuracy of electricity: 0.98
Category:, tools 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      7817
           1       0.00      0.00      0.00        48

   micro avg       0.99      0.99      0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      0.99      0.99      7865

Accuracy of tools: 0.99
Category:, hospitals 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      7787
           1       0.00      0.00      0.00        78

   micro avg       0.99      0.99

### 6. Improve your model
Use grid search to find better parameters. 

In [202]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        #'tfidf__norm' : ['l2', None], 
        'clf__estimator__n_estimators': [50]
    }

model = GridSearchCV(pipeline, param_grid=parameters, cv=3)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [231]:
display_results(Y_test, Y_pred, model)

Category:, related 
               precision    recall  f1-score   support

           0       0.71      0.35      0.47      1873
           1       0.82      0.96      0.88      5934
           2       0.67      0.07      0.12        58

   micro avg       0.80      0.80      0.80      7865
   macro avg       0.73      0.46      0.49      7865
weighted avg       0.79      0.80      0.78      7865

Accuracy of related: 0.80
Category:, request 
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      6533
           1       0.83      0.47      0.60      1332

   micro avg       0.89      0.89      0.89      7865
   macro avg       0.87      0.72      0.77      7865
weighted avg       0.89      0.89      0.88      7865

Accuracy of request: 0.89
Category:, offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00      0.00        36

   micro avg   

Category:, aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      7762
           1       0.00      0.00      0.00       103

   micro avg       0.99      0.99      0.99      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.97      0.99      0.98      7865

Accuracy of aid_centers: 0.99
Category:, other_infrastructure 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      7524
           1       0.00      0.00      0.00       341

   micro avg       0.96      0.96      0.96      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.92      0.96      0.94      7865

Accuracy of other_infrastructure: 0.96
Category:, weather_related 
               precision    recall  f1-score   support

           0       0.88      0.95      0.92      5702
           1       0.85      0.67      0.75      2163

   micro avg       0.88  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [227]:
with open('model.pickle', 'wb') as f:
    pickle.dump(model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.